In [2]:
cd ..

/geode2/home/u020/deduggi/Carbonate/autonomous-robotics/bev-generation


In [23]:
import os
from pathlib import Path
import json
import zlib

import numpy as np

import matplotlib.pyplot as plt

import lmdb # pip install lmdb

import utils
from utils import bytes_to_array

In [6]:
dataroot = Path('/N/slate/deduggi/nuScenes-trainval').resolve()
gt_db_path = dataroot / Path('lmdb/samples/GT_BEV_CAM_FRONT')
raw_db_path = dataroot / Path('lmdb/samples/RAW_BEV_CAM_FRONT')
nusc_metadata_path = './v1.0-trainval-meta-custom.json'

In [8]:
with open(nusc_metadata_path, 'r') as f:
    nusc_metadata = json.load(f)

### Train and Val splits

In [15]:
splits_dir = dataroot / 'splits'
train_split_path = splits_dir / 'train_roddick.txt'
val_split_path = splits_dir / 'val_roddick.txt'

with open(train_split_path, 'r') as f:
    train_samples = f.read().split()
    
with open(val_split_path, 'r') as f:
    val_samples = f.read().split()   
    
len(train_samples), len(val_samples)

(28008, 5981)

In [17]:
sample_token_to_cam_token = {}

for scene in nusc_metadata.values():
    for sample_token, sample in scene['scene_data'].items():
        sample_token_to_cam_token[sample_token] = sample['CAM_FRONT']['token']

### Get GT and raw BEV using the cam data token

In [24]:
gt_db = lmdb.open(path=str(gt_db_path), readonly=True)
raw_db = lmdb.open(path=str(raw_db_path), readonly=True)

In [25]:
def db_value_to_array(value):
    value_unzipped = zlib.decompress(value)
    return bytes_to_array(value_unzipped)

In [26]:
sample_token = train_samples[0]
cam_token = sample_token_to_cam_token[sample_token]
cam_token_bytes = str.encode(cam_token, 'utf-8')
cam_token_bytes

b'8ed2a7ea9a894eac8b3cd43b706f39bc'

In [27]:
with gt_db.begin() as txn:
    value = txn.get(cam_token_bytes)
    bev_gt = db_value_to_array(value)
bev_gt.shape

(15, 196, 200)

In [28]:
labels = utils.transform_label(bev_gt)
labels.shape

(12, 196, 200)

In [29]:
with raw_db.begin() as txn:
    value = txn.get(cam_token_bytes)
    bev_raw = db_value_to_array(value)
bev_raw.shape

(14, 200, 200)

In [30]:
pred = utils.transform_pred(bev_raw)
pred.shape

(12, 196, 200)